Mestrado em Modelagem Matematica da Informacao
----------------------------------------------
Disciplina: Modelagem e Mineracao de Dados
------------------------------------------

Master Program - Mathematical Modeling of Information
-----------------------------------------------------
Course: Data Mining and Modeling
--------------------------------

Professor: Renato Rocha Souza
-----------------------------

### Accessing and Manipulating Data in different formats

In [10]:
import os
import sys
import time
import datetime
import numpy as np
import pandas as pd

In [8]:
datapath = "../datasets/"
outputs = "../outputs/"

## Acessing CSV Files

In [3]:
csvfile = "beatles-diskography.csv"
datafile = os.path.join(datapath, csvfile)

#### Using pure Python:

In [4]:
data = []
with open(datafile, "r") as f:
    keys = f.readline().split(',')
    keys = [k.strip() for k in keys]
    for i in range(5):
        values = f.readline().split(',')
        values = [v.strip() for v in values]
        d = dict(zip(keys,values))
        data.append(d)
data[0:2]

[{'BPI Certification': 'Gold',
  'Label': 'Parlophone(UK)',
  'RIAA Certification': 'Platinum',
  'Released': '22 March 1963',
  'Title': 'Please Please Me',
  'UK Chart Position': '1',
  'US Chart Position': '-'},
 {'BPI Certification': 'Platinum',
  'Label': 'Parlophone(UK)',
  'RIAA Certification': 'Gold',
  'Released': '22 November 1963',
  'Title': 'With the Beatles',
  'UK Chart Position': '1',
  'US Chart Position': '-'}]

#### Using CSV module:

https://docs.python.org/3/library/csv.html  

In [5]:
import csv

In [6]:
with open(datafile, "r") as f:
    #data2 = csv.reader(f)
    data2 = csv.DictReader(f)
    for row in data2:
        print(row)

{'UK Chart Position': '1', 'Title': 'Please Please Me', 'Released': '22 March 1963', 'RIAA Certification': 'Platinum', 'BPI Certification': 'Gold', 'US Chart Position': '-', 'Label': 'Parlophone(UK)'}
{'UK Chart Position': '1', 'Title': 'With the Beatles', 'Released': '22 November 1963', 'RIAA Certification': 'Gold', 'BPI Certification': 'Platinum', 'US Chart Position': '-', 'Label': 'Parlophone(UK)'}
{'UK Chart Position': '-', 'Title': 'Beatlemania! With the Beatles', 'Released': '25 November 1963', 'RIAA Certification': '', 'BPI Certification': '', 'US Chart Position': '-', 'Label': 'Capitol(CAN)'}
{'UK Chart Position': '-', 'Title': 'Introducing... The Beatles', 'Released': '10 January 1964', 'RIAA Certification': '', 'BPI Certification': '', 'US Chart Position': '2', 'Label': 'Vee-Jay(US)'}
{'UK Chart Position': '-', 'Title': 'Meet the Beatles!', 'Released': '20 January 1964', 'RIAA Certification': '5xPlatinum', 'BPI Certification': '', 'US Chart Position': '1', 'Label': 'Capitol(U

#### Using Pandas:

In [7]:
df_csv = pd.read_csv(datafile)
df_csv.head()
#df_csv.info()

,Title,Released,Label,UK Chart Position,US Chart Position,BPI Certification,RIAA Certification
0,Please Please Me,22 March 1963,Parlophone(UK),1,-,Gold,Platinum
1,With the Beatles,22 November 1963,Parlophone(UK),1,-,Platinum,Gold
2,Beatlemania! With the Beatles,25 November 1963,Capitol(CAN),-,-,NaN,NaN
3,Introducing... The Beatles,10 January 1964,Vee-Jay(US),-,2,NaN,NaN
4,Meet the Beatles!,20 January 1964,Capitol(US),-,1,NaN,5xPlatinum


## Manipulating Zip Files

https://docs.python.org/3/library/zipfile.html   

In [8]:
from zipfile import ZipFile

In [9]:
zipfile = "2013_ERCOT_Hourly_Load_Data"

In [15]:
with ZipFile('{0}.zip'.format(os.path.join(datapath,zipfile)), 'r') as myzip:
    myzip.extractall(datapath)

In [16]:
myzip.filename

'../datasets/2013_ERCOT_Hourly_Load_Data.zip'

In [17]:
with ZipFile('{0}.zip'.format(os.path.join(datapath,zipfile)), 'r') as myzip:
    with myzip.open('2013_ERCOT_Hourly_Load_Data.xls') as myfile:
        pd_excel = pd.read_excel(myfile)
pd_excel.head()

,Hour_End,COAST,EAST,FAR_WEST,NORTH,NORTH_C,SOUTHERN,SOUTH_C,WEST,ERCOT
0,2013-01-01 01:00:00,7606.263544,1073.892438,1411.750567,784.978166,10369.094390,2206.675077,4368.490945,882.931901,28704.077028
1,2013-01-01 02:00:00,7388.082714,1035.021938,1403.472287,776.307387,10152.358518,2159.733208,4233.587967,872.404750,28020.968769
2,2013-01-01 03:00:00,7178.867878,1036.088697,1395.053150,768.125748,9988.051418,2065.114706,4082.862860,868.853938,27383.018395
3,2013-01-01 04:00:00,7038.822581,1032.648841,1395.508820,770.937969,9946.658655,1990.903699,4010.489608,865.701201,27051.671374
4,2013-01-01 05:00:00,6990.857940,1042.823044,1401.216842,779.089313,10096.664190,1954.807585,4038.655997,879.924249,27184.039160


## Acessing and Writing Excel Files

https://pypi.python.org/pypi/xlrd  
https://pypi.python.org/pypi/xlwt  
https://github.com/python-excel/tutorial/blob/master/python-excel.pdf  

#### Accessing a workbook

In [18]:
import xlrd

In [19]:
datafile = "2013_ERCOT_Hourly_Load_Data.xls"

In [20]:
workbook = xlrd.open_workbook(os.path.join(datapath,datafile))
sheet = workbook.sheet_by_index(0)

In [21]:
sheet_data = [[sheet.cell_value(r, col) for col in range(sheet.ncols)] for r in range(sheet.nrows)]
sheet_data[:2]

[['Hour_End',
  'COAST',
  'EAST',
  'FAR_WEST',
  'NORTH',
  'NORTH_C',
  'SOUTHERN',
  'SOUTH_C',
  'WEST',
  'ERCOT'],
 [41275.041666666664,
  7606.263544000012,
  1073.892438,
  1411.7505669999982,
  784.9781659999992,
  10369.094390000051,
  2206.6750770000012,
  4368.490945000006,
  882.9319009999975,
  28704.077028000065]]

In [22]:
print("Number of rows in the sheet:"),
print(sheet.nrows)

print("Type of data in cell (row 3, col 2):"), 
print(sheet.cell_type(3, 2))

print("Value in cell (row 3, col 2):", )
print(sheet.cell_value(3, 2))

print("Get a slice of values in column 3, from rows 1-3:",)
print(sheet.col_values(3, start_rowx=1, end_rowx=4))

Number of rows in the sheet:
7296
Type of data in cell (row 3, col 2):
2
Value in cell (row 3, col 2):
1036.0886969999988
Get a slice of values in column 3, from rows 1-3:
[1411.7505669999982, 1403.4722870000019, 1395.053150000001]


In [23]:
coast = sheet.col_values(1, start_rowx=1)
data3 = {}    
data3['maxvalue'] = max(coast)
data3['minvalue'] = min(coast)
data3['avgcoast'] = np.mean(coast)

rowmax = coast.index(max(coast))+1
rowmin = coast.index(min(coast))+1

data3['maxtime'] = xlrd.xldate_as_tuple(sheet.cell_value(rowmax,0), 0)
data3['mintime'] = xlrd.xldate_as_tuple(sheet.cell_value(rowmin,0), 0)

In [24]:
data3

{'avgcoast': 10976.933460679784,
 'maxtime': (2013, 8, 13, 17, 0, 0),
 'maxvalue': 18779.025510000003,
 'mintime': (2013, 2, 3, 4, 0, 0),
 'minvalue': 6602.113898999982}

#### Using Pandas

http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_excel.html  

In [25]:
pd_excel = pd.read_excel(os.path.join(datapath,datafile))
pd_excel.head()

,Hour_End,COAST,EAST,FAR_WEST,NORTH,NORTH_C,SOUTHERN,SOUTH_C,WEST,ERCOT
0,2013-01-01 01:00:00,7606.263544,1073.892438,1411.750567,784.978166,10369.094390,2206.675077,4368.490945,882.931901,28704.077028
1,2013-01-01 02:00:00,7388.082714,1035.021938,1403.472287,776.307387,10152.358518,2159.733208,4233.587967,872.404750,28020.968769
2,2013-01-01 03:00:00,7178.867878,1036.088697,1395.053150,768.125748,9988.051418,2065.114706,4082.862860,868.853938,27383.018395
3,2013-01-01 04:00:00,7038.822581,1032.648841,1395.508820,770.937969,9946.658655,1990.903699,4010.489608,865.701201,27051.671374
4,2013-01-01 05:00:00,6990.857940,1042.823044,1401.216842,779.089313,10096.664190,1954.807585,4038.655997,879.924249,27184.039160


#### Reading a Workbook

In [1]:
import xlwt

In [2]:
style0 = xlwt.easyxf('font: name Times New Roman, color-index red, bold on', num_format_str='#,##0.00')
style1 = xlwt.easyxf(num_format_str='D-MMM-YY')

In [11]:
workbook = xlwt.Workbook()
workbook_name = 'my_workbook.xls'

worksheet1 = workbook.add_sheet('my_first_sheet')
worksheet1.write(0, 0, 'Column 1 title', style0)
worksheet1.write(0, 1, 'Column 2 title', style0)

worksheet2 = workbook.add_sheet('my_second_sheet')
worksheet2.write(0, 0, 'Itens')
worksheet2.write(0, 1, 'Price')
worksheet2.write(1, 0, 'Rice')
worksheet2.write(1, 1, 100)
worksheet2.write(2, 0, 'Beans')
worksheet2.write(2, 1, 200)
worksheet2.write(3, 0, 'Pasta')
worksheet2.write(3, 1, 500)
worksheet2.write(5, 0, 'Total')

worksheet2.write(5, 1, xlwt.Formula("B2+B3+B4"))


workbook.save(os.path.join(outputs, workbook_name))

## Accessing HTML

https://docs.python.org/3/library/urllib.html  
http://www.crummy.com/software/BeautifulSoup/  
http://docs.python-requests.org/en/latest/  
http://lxml.de/  
https://docs.python.org/3/library/getpass.html  

In [26]:
import urllib
from bs4 import BeautifulSoup as bs
import requests
import lxml.html
import getpass

In [27]:
url = "http://oglobo.globo.com"
html_txt = urllib.request.urlopen(url).read()
soup = bs(html_txt, "html.parser")
for line in soup.find_all('a'):
    print(line.get('href'))

http://oglobo.globo.com/
http://memoria.oglobo.globo.com
http://acervo.oglobo.globo.com
http://oglobo.globo.com/ela/
http://kogut.oglobo.globo.com
http://rioshow.oglobo.globo.com/
http://extra.globo.com
http://classificados.globo.com
https://clubesoumaisrio.oglobo.globo.com/?link=og-menu-sup
http://www.centraldoassinante.oglobo.com.br
http://oglobo.globo.com/
http://oglobo.globo.com/principios-editoriais/
http://facebook.com/JornalOGlobo
https://twitter.com/JornalOGlobo
https://plus.google.com/u/0/+JornalOGlobo/posts

https://seguro.oglobo.com.br/assinatura/Default.aspx?id_parc=26&utm_source=venda&utm_medium=botaooglobo&utm_campaign=footer
http://oglobo.globo.com/
http://oglobo.globo.com/rio/
http://oglobo.globo.com/rio/
http://oglobo.globo.com/rio/
http://oglobo.globo.com/rio/ancelmo/
http://oglobo.globo.com/blogs/blog_gente_boa/
http://oglobo.globo.com/rio/carnaval/
http://oglobo.globo.com/rio/bairros/
http://oglobo.globo.com/rio/design-rio/
http://oglobo.globo.com/eu-reporter/
http:

#### Another approach: lxml instead of beautiful soup
http://lxml.de/tutorial.html  

In [22]:
url = "http://www.hotashtanga.com/p/letoltesek-downloads.html"
html_txt = urllib.request.urlopen(url).read()
dom =  lxml.html.fromstring(html_txt)
for link in dom.xpath('//a/@href'): # select the url in href for all a tags(links)
    print(link)

http://www.hotashtanga.com/
http://www.youtube.com/GaurangaDas
https://drive.google.com/file/d/0B0t9QIGAtRQ5Z2VrVl9iYlI0Q0U/edit?usp=sharing
https://drive.google.com/file/d/0B0t9QIGAtRQ5RE9BYkdGTlU2MHc/edit?usp=sharing
http://brihaspati.net/downloads/2009_07_08_Patandzsali_beavatas.mp3
https://drive.google.com/file/d/0B0t9QIGAtRQ5Y2kyb0NpWTgtQnM/edit?usp=sharing
https://drive.google.com/file/d/0B0t9QIGAtRQ5MnFlOXNmN19UMlU/edit?usp=sharing
https://drive.google.com/file/d/0B0t9QIGAtRQ5ellsNDNyOUFEQjA/edit?usp=sharing
https://drive.google.com/file/d/0B0t9QIGAtRQ5bVV4TlNzLTFDZ00/edit?usp=sharing
https://drive.google.com/file/d/0B0t9QIGAtRQ5Ukdoa3V5eE9IM00/edit?usp=sharing
https://drive.google.com/file/d/0B0t9QIGAtRQ5MW1OSjdCNjBUUGc/edit?usp=sharing
https://drive.google.com/file/d/0B0t9QIGAtRQ5RVF2SFNKNnptUzQ/edit?usp=sharing
https://drive.google.com/file/d/0B0t9QIGAtRQ5eWxGUDU5ZzZieDg/edit?usp=sharing
https://drive.google.com/file/d/0B0t9QIGAtRQ5aTN1MkdtNnBISXc/edit?usp=sharing
https://dri

#### Using Requests, for more complex tasks  

http://docs.python-requests.org/en/master/user/quickstart/  
http://www.w3.org/Protocols/rfc2616/rfc2616-sec10.html  

In [23]:
p = getpass.getpass()
r = requests.get('https://api.github.com/user', auth=('rsouza', p))
r.status_code

········


200

In [24]:
r.headers['content-type']

'application/json; charset=utf-8'

In [25]:
r.encoding

'utf-8'

In [26]:
r.text

'{"login":"rsouza","id":604914,"avatar_url":"https://avatars.githubusercontent.com/u/604914?v=3","gravatar_id":"","url":"https://api.github.com/users/rsouza","html_url":"https://github.com/rsouza","followers_url":"https://api.github.com/users/rsouza/followers","following_url":"https://api.github.com/users/rsouza/following{/other_user}","gists_url":"https://api.github.com/users/rsouza/gists{/gist_id}","starred_url":"https://api.github.com/users/rsouza/starred{/owner}{/repo}","subscriptions_url":"https://api.github.com/users/rsouza/subscriptions","organizations_url":"https://api.github.com/users/rsouza/orgs","repos_url":"https://api.github.com/users/rsouza/repos","events_url":"https://api.github.com/users/rsouza/events{/privacy}","received_events_url":"https://api.github.com/users/rsouza/received_events","type":"User","site_admin":false,"name":"Renato Rocha Souza","company":"FGV","blog":"emap.fgv.br","location":"Rio de Janeiro","email":"rsouza.fgv@gmail.com","hireable":null,"bio":"https:

In [27]:
r.json()

{'avatar_url': 'https://avatars.githubusercontent.com/u/604914?v=3',
 'bio': 'https://www.linkedin.com/in/renato-rocha-souza-157153',
 'blog': 'emap.fgv.br',
 'collaborators': 0,
 'company': 'FGV',
 'created_at': '2011-02-07T15:46:45Z',
 'disk_usage': 199458,
 'email': 'rsouza.fgv@gmail.com',
 'events_url': 'https://api.github.com/users/rsouza/events{/privacy}',
 'followers': 22,
 'followers_url': 'https://api.github.com/users/rsouza/followers',
 'following': 5,
 'following_url': 'https://api.github.com/users/rsouza/following{/other_user}',
 'gists_url': 'https://api.github.com/users/rsouza/gists{/gist_id}',
 'gravatar_id': '',
 'hireable': None,
 'html_url': 'https://github.com/rsouza',
 'id': 604914,
 'location': 'Rio de Janeiro',
 'login': 'rsouza',
 'name': 'Renato Rocha Souza',
 'organizations_url': 'https://api.github.com/users/rsouza/orgs',
 'owned_private_repos': 0,
 'plan': {'collaborators': 0,
  'name': 'free',
  'private_repos': 0,
  'space': 976562499},
 'private_gists': 0,

## Accessing JSON

http://json.org/  
https://docs.python.org/3/library/json.html  
https://docs.python.org/3/library/urllib.html  

In [1]:
import json

In [29]:
base_url = "http://musicbrainz.org/ws/2/"
artist_url =  urllib.parse.urljoin(base_url, 'artist/')
query_type = {"simple": {},
              "atr": {"inc": "aliases+tags+ratings"},
              "aliases": {"inc": "aliases"},
              "releases": {"inc": "releases"}}

In [30]:
def query_site(url, params, uid="", fmt="json"):
    params["fmt"] = fmt
    r = requests.get(url + uid, params=params)
    print("requesting", r.url)

    if r.status_code == requests.codes.ok:
        return r.json()
    else:
        r.raise_for_status()

In [31]:
def query_by_name(url, params, name):
    params["query"] = "artist:" + name
    return query_site(url, params)

In [32]:
def pretty_print(data, indent=4):
    if type(data) == dict:
        print(json.dumps(data, indent=indent, sort_keys=True))
    else:
        print(data)

In [33]:
results = query_by_name(artist_url, query_type["simple"], "Nirvana")
pretty_print(results)

requesting http://musicbrainz.org/ws/2/artist/?query=artist%3ANirvana&fmt=json
{
    "artists": [
        {
            "aliases": [
                {
                    "begin-date": null,
                    "end-date": null,
                    "locale": null,
                    "name": "Nirvana US",
                    "primary": null,
                    "sort-name": "Nirvana US",
                    "type": null
                }
            ],
            "area": {
                "id": "489ce91b-6658-3307-9877-795b68554c98",
                "name": "United States",
                "sort-name": "United States"
            },
            "begin-area": {
                "id": "a640b45c-c173-49b1-8030-973603e895b5",
                "name": "Aberdeen",
                "sort-name": "Aberdeen"
            },
            "country": "US",
            "disambiguation": "90s US grunge band",
            "id": "5b11f4ce-a62d-471e-81fc-a69a8278c7da",
            "life-span": {
           

In [34]:
results.keys()

dict_keys(['offset', 'count', 'created', 'artists'])

In [35]:
artist_id = results["artists"][1]["id"]
artist_id

'9282c8b4-ca0b-4c6b-b7e3-4f7762dfc4d6'

In [36]:
print("\nARTIST:")
pretty_print(results["artists"][0])


ARTIST:
{
    "aliases": [
        {
            "begin-date": null,
            "end-date": null,
            "locale": null,
            "name": "Nirvana US",
            "primary": null,
            "sort-name": "Nirvana US",
            "type": null
        }
    ],
    "area": {
        "id": "489ce91b-6658-3307-9877-795b68554c98",
        "name": "United States",
        "sort-name": "United States"
    },
    "begin-area": {
        "id": "a640b45c-c173-49b1-8030-973603e895b5",
        "name": "Aberdeen",
        "sort-name": "Aberdeen"
    },
    "country": "US",
    "disambiguation": "90s US grunge band",
    "id": "5b11f4ce-a62d-471e-81fc-a69a8278c7da",
    "life-span": {
        "begin": "1988-01",
        "end": "1994-04-05",
        "ended": true
    },
    "name": "Nirvana",
    "score": "100",
    "sort-name": "Nirvana",
    "tags": [
        {
            "count": 1,
            "name": "punk"
        },
        {
            "count": 0,
            "name": "legendary"

In [37]:
artist_data = query_site(artist_url, query_type["releases"], artist_id)
releases = artist_data["releases"]
print("\nONE RELEASE:")
pretty_print(releases[0], indent=2)

requesting http://musicbrainz.org/ws/2/artist/9282c8b4-ca0b-4c6b-b7e3-4f7762dfc4d6?fmt=json&inc=releases

ONE RELEASE:
{
  "barcode": null,
  "country": "GB",
  "date": "1969",
  "disambiguation": "",
  "id": "0b44cb36-550a-491d-bfd9-8751271f9de7",
  "packaging": null,
  "packaging-id": null,
  "quality": "normal",
  "release-events": [
    {
      "area": {
        "disambiguation": "",
        "id": "8a754a16-0027-3a29-b6d7-2b40ea0481ed",
        "iso-3166-1-codes": [
          "GB"
        ],
        "name": "United Kingdom",
        "sort-name": "United Kingdom"
      },
      "date": "1969"
    }
  ],
  "status": "Official",
  "status-id": "4e304316-386d-3409-af2e-78857eec5cfe",
  "text-representation": {
    "language": "eng",
    "script": "Latn"
  },
  "title": "To Markos III"
}


In [38]:
releases

[{'barcode': None,
  'country': 'GB',
  'date': '1969',
  'disambiguation': '',
  'id': '0b44cb36-550a-491d-bfd9-8751271f9de7',
  'packaging': None,
  'packaging-id': None,
  'quality': 'normal',
  'release-events': [{'area': {'disambiguation': '',
     'id': '8a754a16-0027-3a29-b6d7-2b40ea0481ed',
     'iso-3166-1-codes': ['GB'],
     'name': 'United Kingdom',
     'sort-name': 'United Kingdom'},
    'date': '1969'}],
  'status': 'Official',
  'status-id': '4e304316-386d-3409-af2e-78857eec5cfe',
  'text-representation': {'language': 'eng', 'script': 'Latn'},
  'title': 'To Markos III'},
 {'barcode': None,
  'country': 'GB',
  'date': '1992',
  'disambiguation': '',
  'id': '223a7069-3d54-40a1-9e57-fc1339f38aee',
  'packaging': None,
  'packaging-id': None,
  'quality': 'normal',
  'release-events': [{'area': {'disambiguation': '',
     'id': '8a754a16-0027-3a29-b6d7-2b40ea0481ed',
     'iso-3166-1-codes': ['GB'],
     'name': 'United Kingdom',
     'sort-name': 'United Kingdom'},
    

In [39]:
release_titles = [r["title"] for r in releases]
print("\nALL TITLES:")
for t in release_titles:
        print(t)


ALL TITLES:
To Markos III
Travelling on a Cloud
Songs Of Love And Praise
Songs of Love and Praise
Songs of Love and Praise
Secret Theatre
The Story of Simon Simopath
Me And My Friend
All of Us
The Story of Simon Simopath
To Markos III
Chemistry
Local Anaesthetic
Orange & Blue
Pentecost Hotel
Black Flower
All of Us


#### Using Pandas

http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader  

In [40]:
df_json = pd.read_json('https://api.github.com/repos/pydata/pandas/issues?per_page=5')
df_json

,assignee,assignees,body,closed_at,comments,comments_url,created_at,events_url,html_url,id,...,locked,milestone,number,pull_request,repository_url,state,title,updated_at,url,user
0,NaN,[],"#### Code Sample, a copy-pastable example if p...",NaT,0,https://api.github.com/repos/pydata/pandas/iss...,2016-07-18 15:29:17,https://api.github.com/repos/pydata/pandas/iss...,https://github.com/pydata/pandas/issues/13694,166121686,...,False,None,13694,NaN,https://api.github.com/repos/pydata/pandas,open,to_excel function not working as expected,2016-07-18 15:29:17,https://api.github.com/repos/pydata/pandas/iss...,"{'site_admin': False, 'type': 'User', 'login':..."
1,NaN,[],- [x] closes 13652\r\n - [x] tests added / pa...,NaT,3,https://api.github.com/repos/pydata/pandas/iss...,2016-07-18 14:43:01,https://api.github.com/repos/pydata/pandas/iss...,https://github.com/pydata/pandas/pull/13693,166110527,...,False,None,13693,{'url': 'https://api.github.com/repos/pydata/p...,https://api.github.com/repos/pydata/pandas,open,"Test case for patch, plus fix to not swallow e...",2016-07-18 16:09:36,https://api.github.com/repos/pydata/pandas/iss...,"{'site_admin': False, 'type': 'User', 'login':..."
2,NaN,[],"- [x] closes #11169, closes #11287\r\n - [x] ...",NaT,1,https://api.github.com/repos/pydata/pandas/iss...,2016-07-18 12:57:41,https://api.github.com/repos/pydata/pandas/iss...,https://github.com/pydata/pandas/pull/13692,166087406,...,False,"{'number': 40, 'closed_at': None, 'labels_url'...",13692,{'url': 'https://api.github.com/repos/pydata/p...,https://api.github.com/repos/pydata/pandas,open,PERF: improve DTI string parse,2016-07-18 13:46:39,https://api.github.com/repos/pydata/pandas/iss...,"{'site_admin': False, 'type': 'User', 'login':..."
3,NaN,[],"#### Code Sample, a copy-pastable example if p...",NaT,0,https://api.github.com/repos/pydata/pandas/iss...,2016-07-18 12:46:10,https://api.github.com/repos/pydata/pandas/iss...,https://github.com/pydata/pandas/issues/13691,166085272,...,False,None,13691,NaN,https://api.github.com/repos/pydata/pandas,open,s.loc[[]] raises error (only) if index is not ...,2016-07-18 12:46:26,https://api.github.com/repos/pydata/pandas/iss...,"{'site_admin': False, 'type': 'User', 'login':..."
4,NaN,[],- [x] closes #13598 \r\n - [x] tests added / ...,NaT,3,https://api.github.com/repos/pydata/pandas/iss...,2016-07-17 19:24:26,https://api.github.com/repos/pydata/pandas/iss...,https://github.com/pydata/pandas/pull/13690,165991394,...,False,"{'number': 40, 'closed_at': None, 'labels_url'...",13690,{'url': 'https://api.github.com/repos/pydata/p...,https://api.github.com/repos/pydata/pandas,open,BUG: Add type check for width parameter in st...,2016-07-18 03:47:06,https://api.github.com/repos/pydata/pandas/iss...,"{'site_admin': False, 'type': 'User', 'login':..."


## Accessing XML

https://docs.python.org/3/library/urllib.html

https://docs.python.org/3/library/xml.etree.elementtree.html

http://lxml.de/

http://www.w3schools.com/xml/xml_examples.asp

In [41]:
from xml.dom import minidom
#from xml.etree import ElementTree as ET
from lxml import etree as ET #Supports xpath syntax
from collections import defaultdict
from pprint import pprint

In [42]:
url = 'http://www.w3schools.com/xml/cd_catalog.xml'
xml_page = urllib.request.urlopen(url).read()
e = ET.XML(xml_page)

In [43]:
def etree_to_dict(t):
    d = {t.tag: {} if t.attrib else None}
    children = list(t)
    if children:
        dd = defaultdict(list)
        for dc in map(etree_to_dict, children):
            for k, v in dc.items():
                dd[k].append(v)
        d = {t.tag: {k:v[0] if len(v) == 1 else v for k, v in dd.items()}}
    if t.attrib:
        d[t.tag].update(('@' + k, v) for k, v in t.attrib.items())
    if t.text:
        text = t.text.strip()
        if children or t.attrib:
            if text:
              d[t.tag]['#text'] = text
        else:
            d[t.tag] = text
    return d

In [44]:
pprint(etree_to_dict(e))

{'CATALOG': {'CD': [{'ARTIST': 'Bob Dylan',
                     'COMPANY': 'Columbia',
                     'COUNTRY': 'USA',
                     'PRICE': '10.90',
                     'TITLE': 'Empire Burlesque',
                     'YEAR': '1985'},
                    {'ARTIST': 'Bonnie Tyler',
                     'COMPANY': 'CBS Records',
                     'COUNTRY': 'UK',
                     'PRICE': '9.90',
                     'TITLE': 'Hide your heart',
                     'YEAR': '1988'},
                    {'ARTIST': 'Dolly Parton',
                     'COMPANY': 'RCA',
                     'COUNTRY': 'USA',
                     'PRICE': '9.90',
                     'TITLE': 'Greatest Hits',
                     'YEAR': '1982'},
                    {'ARTIST': 'Gary Moore',
                     'COMPANY': 'Virgin records',
                     'COUNTRY': 'UK',
                     'PRICE': '10.20',
                     'TITLE': 'Still got the blues',
                

In [45]:
raw_page = urllib.request.urlopen(url)
doc = minidom.parse(raw_page)

def findTextnodes(nodeList):
    for subnode in nodeList:
        if subnode.nodeType == subnode.ELEMENT_NODE:
            print("Element node: " + subnode.tagName)
            findTextnodes(subnode.childNodes)
        elif subnode.nodeType == subnode.TEXT_NODE:
            print("text node:" + subnode.data)
            
findTextnodes(doc.childNodes)

Element node: CATALOG
text node:
	
Element node: CD
text node:
		
Element node: TITLE
text node:Empire Burlesque
text node:
		
Element node: ARTIST
text node:Bob Dylan
text node:
		
Element node: COUNTRY
text node:USA
text node:
		
Element node: COMPANY
text node:Columbia
text node:
		
Element node: PRICE
text node:10.90
text node:
		
Element node: YEAR
text node:1985
text node:
	
text node:
	
Element node: CD
text node:
		
Element node: TITLE
text node:Hide your heart
text node:
		
Element node: ARTIST
text node:Bonnie Tyler
text node:
		
Element node: COUNTRY
text node:UK
text node:
		
Element node: COMPANY
text node:CBS Records
text node:
		
Element node: PRICE
text node:9.90
text node:
		
Element node: YEAR
text node:1988
text node:
	
text node:
	
Element node: CD
text node:
		
Element node: TITLE
text node:Greatest Hits
text node:
		
Element node: ARTIST
text node:Dolly Parton
text node:
		
Element node: COUNTRY
text node:USA
text node:
		
Element node: COMPANY
text node:RCA
text 

In [46]:
root = ET.fromstring(xml_page)
cdtags = root.xpath('//CD/TITLE')
for cd in cdtags:
    print(cd.text)

Empire Burlesque
Hide your heart
Greatest Hits
Still got the blues
Eros
One night only
Sylvias Mother
Maggie May
Romanza
When a man loves a woman
Black angel
1999 Grammy Nominees
For the good times
Big Willie style
Tupelo Honey
Soulsville
The very best of
Stop
Bridge of Spies
Private Dancer
Midt om natten
Pavarotti Gala Concert
The dock of the bay
Picture book
Red
Unchain my heart


In [47]:
for cd in root.findall('CD'):
    title = cd.find('TITLE').text
    artist = cd.find('ARTIST').text
    country = cd.find('COUNTRY').text
    print('{}\t{}\t{}'.format(artist, country, title))

Bob Dylan	USA	Empire Burlesque
Bonnie Tyler	UK	Hide your heart
Dolly Parton	USA	Greatest Hits
Gary Moore	UK	Still got the blues
Eros Ramazzotti	EU	Eros
Bee Gees	UK	One night only
Dr.Hook	UK	Sylvias Mother
Rod Stewart	UK	Maggie May
Andrea Bocelli	EU	Romanza
Percy Sledge	USA	When a man loves a woman
Savage Rose	EU	Black angel
Many	USA	1999 Grammy Nominees
Kenny Rogers	UK	For the good times
Will Smith	USA	Big Willie style
Van Morrison	UK	Tupelo Honey
Jorn Hoel	Norway	Soulsville
Cat Stevens	UK	The very best of
Sam Brown	UK	Stop
T'Pau	UK	Bridge of Spies
Tina Turner	UK	Private Dancer
Kim Larsen	EU	Midt om natten
Luciano Pavarotti	UK	Pavarotti Gala Concert
Otis Redding	USA	The dock of the bay
Simply Red	EU	Picture book
The Communards	UK	Red
Joe Cocker	USA	Unchain my heart


In [48]:
cds = []
for cd in root.findall('./CD'):
        data = {}
        data["TITLE"] = cd.find('./TITLE').text
        data["ARTIST"] = cd.find('./ARTIST').text
        data["COUNTRY"] = cd.find('./COUNTRY').text
        data["PRICE"] = cd.find('./PRICE').text
        data["YEAR"] = cd.find('./YEAR').text
        cds.append(data)
cds[0:3]

[{'ARTIST': 'Bob Dylan',
  'COUNTRY': 'USA',
  'PRICE': '10.90',
  'TITLE': 'Empire Burlesque',
  'YEAR': '1985'},
 {'ARTIST': 'Bonnie Tyler',
  'COUNTRY': 'UK',
  'PRICE': '9.90',
  'TITLE': 'Hide your heart',
  'YEAR': '1988'},
 {'ARTIST': 'Dolly Parton',
  'COUNTRY': 'USA',
  'PRICE': '9.90',
  'TITLE': 'Greatest Hits',
  'YEAR': '1982'}]

## Acessing Databases

### Acessing SQLite

https://docs.python.org/3/library/sqlite3.html

In [28]:
import sqlite3
conn = sqlite3.connect(os.path.join(outputs,'example.db'))

In [29]:
c = conn.cursor()

# Create table
c.execute('''CREATE TABLE IF NOT EXISTS stocks
             (date text, trans text, symbol text, qty real, price real)''')

# Insert a row of data
c.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT',100,35.14)")

# Save (commit) the changes
conn.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()

In [30]:
conn = sqlite3.connect(os.path.join(outputs,'example.db'))
c = conn.cursor()

In [54]:
t = ('RHAT',) #tuple with just one element
c.execute('SELECT * FROM stocks WHERE symbol=?', t)
print(c.fetchone())

('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)


In [55]:
# Larger example that inserts many records at a time
purchases = [('2006-03-28', 'BUY', 'IBM', 1000, 45.00),
             ('2006-04-05', 'BUY', 'MSFT', 1000, 72.00),
             ('2006-04-06', 'SELL', 'IBM', 500, 53.00),
            ]
c.executemany('INSERT INTO stocks VALUES (?,?,?,?,?)', purchases)

In [56]:
for row in c.execute('SELECT * FROM stocks ORDER BY price'):
        print(row)

('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)
('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)
('2006-03-28', 'BUY', 'IBM', 1000.0, 45.0)
('2006-04-06', 'SELL', 'IBM', 500.0, 53.0)
('2006-04-05', 'BUY', 'MSFT', 1000.0, 72.0)


#### Using Pandas

In [101]:
conn = sqlite3.connect(os.path.join(outputs,'example.db'))
df = pd.read_sql_query("SELECT * from stocks ORDER BY price", conn)

# verify that result of SQL query is stored in the dataframe
print(df.head())

conn.close()

         date trans symbol    qty  price
0  2006-01-05   BUY   RHAT  100.0  35.14
1  2006-01-05   BUY   RHAT  100.0  35.14


### Acessing MySQL  

https://pypi.python.org/pypi/PyMySQL  

In [57]:
import pymysql

In [61]:
p = getpass.getpass()
conn = pymysql.connect(host='localhost', port=3306, user='root', passwd=p,)# db='example')
cur = conn.cursor()

········


In [62]:
cur.execute("show databases;")
for r in cur.fetchall():
   print(r)

('information_schema',)
('mysql',)
('performance_schema',)
('sys',)


In [63]:
cur.execute("use mysql;")
cur.execute("show tables;")
for r in cur.fetchall():
   print(r)

('columns_priv',)
('db',)
('engine_cost',)
('event',)
('func',)
('general_log',)
('gtid_executed',)
('help_category',)
('help_keyword',)
('help_relation',)
('help_topic',)
('innodb_index_stats',)
('innodb_table_stats',)
('ndb_binlog_index',)
('plugin',)
('proc',)
('procs_priv',)
('proxies_priv',)
('server_cost',)
('servers',)
('slave_master_info',)
('slave_relay_log_info',)
('slave_worker_info',)
('slow_log',)
('tables_priv',)
('time_zone',)
('time_zone_leap_second',)
('time_zone_name',)
('time_zone_transition',)
('time_zone_transition_type',)
('user',)


In [75]:
cur.execute("CREATE DATABASE IF NOT EXISTS test;")
cur.execute("USE test;")
cur.execute('''CREATE TABLE IF NOT EXISTS test_table 
               (text VARCHAR(200)
               DEFAULT NULL)
               ENGINE=MyISAM DEFAULT CHARSET='utf8';''')

cur.execute("show tables;")
for r in cur.fetchall():
   print(r)

('test_table',)


/home/rsouza/python/3/venv/local/lib/python3.5/site-packages/pymysql/cursors.py:161: Warning: Can't create database 'test'; database exists
  result = self._query(query)


In [76]:
texts = ['It always seems impossible until its done.',
         'In order to succeed, we must first believe that we can.',
         'Life is 10% what happens to you and 90% how you react to it.',
         'Start where you are. Use what you have. Do what you can.',]
cur.executemany('INSERT INTO test.test_table (text) VALUES (%s)', texts)



4

In [77]:
cur.execute("SELECT * FROM test.test_table;")# LIMIT 10;")
for r in cur.fetchall():
   print(r)

('It always seems impossible until its done.',)
('In order to succeed, we must first believe that we can.',)
('Life is 10% what happens to you and 90% how you react to it.',)
('Start where you are. Use what you have. Do what you can.',)


#### Using Pandas

In [78]:
import pandas.io.sql as psql

In [79]:
df_mysql = psql.read_sql('select * from test.test_table;', con=conn)

In [80]:
df_mysql.head()

,text
0,It always seems impossible until its done.
1,"In order to succeed, we must first believe tha..."
2,Life is 10% what happens to you and 90% how yo...
3,Start where you are. Use what you have. Do wha...


In [81]:
cur.close()
conn.close()

### Acessing PostgreSQL 

In [82]:
import psycopg2

In [89]:
p = getpass.getpass()
try:
    conn = psycopg2.connect("dbname='cablegate' user='rsouza' host='localhost' password='{}'".format(p))
except:
    print("I am unable to connect to the database")
cur = conn.cursor()

········


In [93]:
cur.execute('SELECT * FROM cable LIMIT 2;')
for r in cur.fetchall():
   print(r)

(1, datetime.datetime(1966, 12, 28, 0, 0), '66BUENOSAIRES2481', 'UNCLASSIFIED', 'Embassy Buenos Aires', '66STATE106206', 'P R 281848Z DEC 66\nFM AMEMBASSY BUENOS AIRES\nTO SECSTATE WASHDC PRIORITY\nINFO USCINCSO\nCINCLANT\nAMEMBASSY RIO DE JANEIRO\nAMEMBASSY SANTIAGO\nAMEMBASSY MONTEVIDEO\nAMEMBASSY QUITO\nAMEMBASSY LIMA\nAMEMBASSY MEXICO\nAMEMBASSY OTTAWA\nAMEMBASSY LONDON\nSTATE GRNC', '\nUNCLASSIFIED BUENOS AIRES 2481 \n \nOriginal Telegram was Confidential but has since been \nde-classified \n \n--------------------------------------------- ---- \nCopy from the National Archives \nRG 59: General Records of the Department of state \n1964-66 Central Foreign Policy File \nFile: POL 33-4 ARG \n--------------------------------------------- ---- \n \nE.O. 12958: DECL: DECLASSIFIED BY NARA 09/02/2009 \nTAGS: EFIS, PBTS, AR \nSUBJECT:  EXTENDED NATIONAL JURISDICTIONS OVER HIGH SEAS \n \nREF: STATE 106206 CIRCULAR; STATE CA-3400 NOV 2, 1966 \n \n \n1.  PRESS REPORTS AND VARIETY EMBASSY SOUR

#### Using Pandas

In [102]:
p = getpass.getpass()
try:
    conn = psycopg2.connect("dbname='cablegate' user='rsouza' host='localhost' password='{}'".format(p))
except:
    print("I am unable to connect to the database")

········


In [104]:
df_postgres = psql.read_sql('SELECT classification, origin, destination FROM cable LIMIT 5;', con=conn)
df_postgres.head()

,classification,origin,destination
0,UNCLASSIFIED,Embassy Buenos Aires,66STATE106206
1,UNCLASSIFIED,Embassy Tehran,72MOSCOW1603|72TEHRAN1091|72TEHRAN263
2,UNCLASSIFIED,Embassy Tehran,
3,UNCLASSIFIED,Embassy Tehran,72TEHRAN3312
4,UNCLASSIFIED,Embassy Tehran,72TEHRAN4887


### Acessing Mongo DB

http://api.mongodb.org/python/current/tutorial.html

In [126]:
from pymongo import MongoClient

In [148]:
client = MongoClient()
#client = MongoClient('localhost', 27017)
db = client.test_database #acessa ou cria o banco

## dropping a database via pymongo
#client.drop_database('test_database')

In [149]:
post = {"author": "Renato",
        "text": "My first blog post!",
        "tags": ["mongodb", "python", "pymongo"],
        "date": datetime.datetime.utcnow()}

In [150]:
posts = db.posts # cria a coleção
post_id = posts.insert_one(post) #insere dados
post_id

## drop a collection via pymongo
#client['test_database'].drop_collection('posts')

In [151]:
db.collection_names()

['posts', 'system.indexes']

In [152]:
posts.find_one()

{'_id': ObjectId('578d17babb7d8c3661329adf'),
 'author': 'Renato',
 'date': datetime.datetime(2016, 7, 18, 17, 54, 1, 212000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}

In [153]:
new_posts = [{"author": "Mike",
              "text": "Another post!",
              "tags": ["bulk", "insert"],
              "date": datetime.datetime(2009, 11, 12, 11, 14)},
             {"author": "Eliot",
              "title": "MongoDB is fun",
              "text": "and pretty easy too!",
              "date": datetime.datetime(2009, 11, 10, 10, 45)}]

In [154]:
posts.insert_many(new_posts)

In [155]:
posts.find_one({"author": "Mike"})

{'_id': ObjectId('578d17bebb7d8c3661329ae0'),
 'author': 'Mike',
 'date': datetime.datetime(2009, 11, 12, 11, 14),
 'tags': ['bulk', 'insert'],
 'text': 'Another post!'}

In [156]:
#for post in posts.find({"author": "Mike"}):
for post in posts.find():
    print(post)

{'author': 'Renato', 'date': datetime.datetime(2016, 7, 18, 17, 54, 1, 212000), 'tags': ['mongodb', 'python', 'pymongo'], 'text': 'My first blog post!', '_id': ObjectId('578d17babb7d8c3661329adf')}
{'author': 'Mike', 'date': datetime.datetime(2009, 11, 12, 11, 14), 'tags': ['bulk', 'insert'], 'text': 'Another post!', '_id': ObjectId('578d17bebb7d8c3661329ae0')}
{'author': 'Eliot', 'date': datetime.datetime(2009, 11, 10, 10, 45), 'title': 'MongoDB is fun', '_id': ObjectId('578d17bebb7d8c3661329ae1'), 'text': 'and pretty easy too!'}


In [157]:
posts.count()

3

#### Using Pandas

In [158]:
example_pd = posts.find()
df_pandas =  pd.DataFrame(list(example_pd))
df_pandas

,_id,author,date,tags,text,title
0,578d17babb7d8c3661329adf,Renato,2016-07-18 17:54:01.212,"[mongodb, python, pymongo]",My first blog post!,NaN
1,578d17bebb7d8c3661329ae0,Mike,2009-11-12 11:14:00.000,"[bulk, insert]",Another post!,NaN
2,578d17bebb7d8c3661329ae1,Eliot,2009-11-10 10:45:00.000,NaN,and pretty easy too!,MongoDB is fun


## Accessing Feeds RSS

http://www.feedparser.org/  
http://docs.python.org/library/re.html  
http://www.pythonware.com/library/pil/handbook/index.htm  

In [160]:
import feedparser

In [161]:
#d = feedparser.parse('http://g1.globo.com/dynamo/rss2.xml')
d = feedparser.parse('http://rss.nytimes.com/services/xml/rss/nyt/InternationalHome.xml')

In [163]:
print(d['feed']['title'])
print(d['feed']['link'])
print(d.feed.subtitle)
print(len(d['entries']))

NYT > International Home
http://www.nytimes.com/pages/international/index.html?partner=rss&emc=rss

36


In [164]:
print(d['entries'][0]['title']) 
print(d.entries[0]['link'])

Baton Rouge Gunman ‘Was Targeting Officers,’ Police Say
http://www.nytimes.com/2016/07/19/us/baton-rouge-shooting.html?partner=rss&emc=rss


In [165]:
print(d.headers)

{'Expires': 'Mon, 25 Jul 2016 17:55:03 GMT', 'Content-Encoding': 'gzip', 'Accept-Ranges': 'bytes', 'Last-Modified': 'Mon, 18 Jul 2016 17:45:25 GMT', 'Connection': 'close', 'Content-Type': 'text/xml', 'Date': 'Mon, 18 Jul 2016 17:55:03 GMT', 'Vary': 'Accept-Encoding', 'Cache-Control': 'max-age=604800, public', 'X-API-Version': '4', 'Server': 'Apache', 'Content-Length': '11877'}


In [166]:
for post in d.entries:
    print(post.title + ": " + post.link + "\n")

Baton Rouge Gunman ‘Was Targeting Officers,’ Police Say: http://www.nytimes.com/2016/07/19/us/baton-rouge-shooting.html?partner=rss&emc=rss

: http://www.nytimes.com/interactive/2016/07/17/us/what-happened-in-the-shooting-of-police-officers-in-baton-rouge.html?partner=rss&emc=rss

Baton Rouge Attack Deepens Anguish for Police: ‘We’ve Seen Nothing Like This’: http://www.nytimes.com/2016/07/18/us/baton-rouge-attack-deepens-anguish-for-police-weve-seen-nothing-like-this.html?partner=rss&emc=rss

Baton Rouge Police Shooting: What We Know: http://www.nytimes.com/2016/07/18/us/baton-rouge-police-shooting-what-we-know-and-do-not-know.html?partner=rss&emc=rss

What Is a Constant Cycle of Violent News Doing to Us?: http://www.nytimes.com/2016/07/16/health/what-is-a-constant-cycle-of-violent-news-doing-to-us.html?partner=rss&emc=rss

Another Baltimore Police Officer Acquitted in Freddie Gray Case: http://www.nytimes.com/2016/07/19/us/freddie-gray-baltimore-police.html?partner=rss&emc=rss

Report

## Acessando Twitter

https://github.com/bear/python-twitter  
Go to http://twitter.com/apps/new to create an app and get these items  
See https://dev.twitter.com/docs/auth/oauth for more information on Twitter's OAuth implementation  
https://dev.twitter.com/rest/reference/get/account/verify_credentials  
https://dev.twitter.com/oauth/overview/application-owner-access-tokens  

In [167]:
import twitter

In [169]:
with open('/home/rsouza/Dropbox/Renato/ModMinDados/Git/notebooks/twitter_tokens.txt', 'r') as twitter_tokens:
    tokens = twitter_tokens.read().split(',')
consumer_key = tokens[0].strip()
consumer_secret = tokens[1].strip()
access_token = tokens[2].strip()
access_token_secret = tokens[3].strip()

Construct, sign, and open a twitter request using the hard-coded credentials above.

In [170]:
api = twitter.Api(consumer_key=consumer_key, consumer_secret=consumer_secret, 
                  access_token_key=access_token, access_token_secret=access_token_secret)

In [172]:
print(api.VerifyCredentials())

{"created_at": "Thu Apr 09 10:44:07 +0000 2009", "description": "Researcher and Professor", "favourites_count": 1, "followers_count": 496, "friends_count": 132, "id": 29959702, "lang": "en", "listed_count": 20, "location": "Rio de Janeiro", "name": "Renato Rocha Souza", "profile_background_color": "9AE4E8", "profile_background_image_url": "http://pbs.twimg.com/profile_background_images/33941499/8.jpg", "profile_image_url": "http://pbs.twimg.com/profile_images/503088023/Renato2_normal.JPG", "profile_link_color": "0084B4", "profile_sidebar_fill_color": "DDFFCC", "profile_text_color": "333333", "screen_name": "rrsouza", "status": {"created_at": "Mon Jan 05 11:05:11 +0000 2015", "favorite_count": 1, "hashtags": [{"text": "FGV"}, {"text": "emap"}], "id": 552058220781797376, "id_str": "552058220781797376", "lang": "und", "source": "<a href=\"http://twitter.com\" rel=\"nofollow\">Twitter Web Client</a>", "text": "http://t.co/5l4lGVKiBG #FGV #emap", "urls": [{"expanded_url": "http://blog.udaci

In [173]:
users = api.GetFriends()
print([u.name for u in users])

['Carla Codeço Coelho', 'Overleaf', 'Cláudio Berrondo', 'Flávio Amieiro', 'Pablo Cerdeira', 'Cut the Rope', 'Machinalis', 'MiningTheSocialWeb', 'CNET News', 'ReadWrite', 'The Next Web', 'CNET', 'The Verge', 'Engadget', 'Lifehacker', 'Gizmodo', 'WIRED', 'TechCrunch', 'Ian Ozsvald', 'CNPq', 'Claudio Gnoli', 'strongsteam', 'Diana Maynard', 'Joyce Figueiró', 'Álvaro Justen', 'PyPLN', 'Open Source For You', 'TechRepublic', 'ZDNet', 'Twitter API', 'Udacity', 'Clarissa', 'Roberto R.S.Sobrinho', 'Luana Sales', 'Sensacionalista', 'André Branco', 'CMA', 'Asla Sá', 'Renata Teixeira', 'FGV EMAp', 'FGV', 'FGV Marketing', 'CPDOC', 'Globant Brasil', 'Sheffield NLP', 'Débora Pereira', 'André Andrade', "O'Reilly Radar", 'Virgilio Almeida', 'Dalai Lama', 'Alexandre Colucci', 'Benildes Maculan', 'Gabriel Rezende', 'Mashable', 'alfredo hirsch', 'Fernando S Parreiras', 'NYTimes Bits', 'MiniCursos', 'Googleverse', 'Hootsuite', 'G1 - Concursos', 'Ivana', 'Dina Araujo', 'João Batista', 'W3C Brasil', 'ASIS&T',

In [174]:
followers = api.GetFollowers()
print([u.screen_name for u in followers])

['DataScienceFest', 'cristinabg6', 'flor322003', 'Bizarro_Pedro', 'klaussherzog', 'gwyngarrison1', 'TTCSP', 'RosariaOtoni', 'shiki_09', 'ColumbiaGPI', 'karusimionato', 'icio_fabr', 'mafradenise', 'chicamarrenta', 'Neurociencia', 'Renato_dxter', 'MMaurosilvareis', 'friedadavis84', 'celiadiamantino', 'pamelabmachado', 'gabyprevidello', 'Stela_Maris_CS', 'FernandoKenned8', 'SandraValeriacd', 'w2amaral', 'Dinieledfisica', 'FlorVsoares', 'KeilaRosado', 'Binoculocultura', 'swordjack', 'Krisluz32', 'klawasaki', 'jujudesouza2', 'MgDayanne', 'ignaciojrctp', 'mrbastos_', 'flavioamieiro', 'dayfamaral', 'llira5', 'SilviaPontes3', 'liara_santos', 'Mmdeoliver', 'Jessika18Leal', 'Airton_Costa', 'angelo_m_calvao', 'macfigueiredo', 'linhares', 'MarcelaRuas', 'thiagon52259992', 'heraldoborges', 'luizclaudioprof', 'cristyan545', '2014Jeh', 'lelebrito1', 'NAMD_EMAp', 'roseanevinte', 'murilogentil', 'gwtostes', 'joquizansouzafr', 'PfcoelhosPaulo', 'cmcf71', 'KatharineRosser', 'FelipeASantos', 'lindajheyne'

In [182]:
mentions = api.GetMentions()
#mentions[0].text
print([m.text for m in mentions])

['@rrsouza \nAre you interested in methods of spiritual development? https://t.co/HB7TO8NmUr', 'Hey twitter... estou de volta para dizer que QUERIA MUITO QUE MEU MESTRADO CRESCESSE E SE DEFENDESSE SOZINHO!! #DESESPERO @rrsouza', 'Diário Thiago Bittencourt is out! http://t.co/Bc2Tjro2WY Stories via @rrsouza @Portal_HD', 'Poster approved on @EuroSciPy 2013! \\o/ https://t.co/wPfbZi71fe Brussels, here we go! #Python #dev //cc @flavioamieiro @fccoelho @rrsouza', 'Many thanks @rrsouza and all #ISKOBrazil people for assistance with the video keynote! It was a fun experience...', '@pablocerdeira @rrsouza Competência analítica não é "feature" de Bancos de dados, mas de pessoas!', '@rrsouza Python gets big data boost darpa http://t.co/5cruMEdd', 'RT @rrsouza: Resultados da seleção do Mestrado em Modelagem Matemática da Informação - turma 2013 saem nesta sexta, 08/02. #FGV #EMAp', '"Turicas é uma espécie de golfinho?" -- @rrsouza depois de ver meu último slide na palestra #Python #Unicode http:/

In [184]:
statuses = api.GetUserTimeline(screen_name='TIME')
print([s.text for s in statuses])

['Watch John Oliver make a few endorsements of his own https://t.co/DcKlkVn9iX', "Hillary Clinton decries 'madness' after murder of Baton Rouge cops https://t.co/3SMG49aMmA", "Chris Pratt achieves dream of blowing stuff up in the trailer for 'The Magnificent Seven' https://t.co/sCG9Ndi7my", "Chris Hemsworth is coming back for yet another 'Star Trek' movie https://t.co/mN8mjRoM2m", '"Trump had one \'yuge\' reason for picking Mike Pence" https://t.co/rlXvwBpHrm', 'Plenty of judges besides Ruth Bader Ginsburg speak out https://t.co/yuqC3XWluE', "HBO confirms that Game of Thrones season 7 won't come nearly soon enough https://t.co/B7wFTLhVOK", 'This is the favorite food of Republicans https://t.co/L1djGaTMM4', 'This is the coolest drone photo in the world https://t.co/qMn4FDPCJN', 'See how a 77-year-old astronaut has inspired a new generation https://t.co/8PuGGwF4Z7 https://t.co/Z5o4PLpvGv', 'A new Zika case in Utah has experts stumped https://t.co/wzPAy1fuA0', "Paul Ryan's overwhelmingly 